In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
#read data file
df = pd.read_csv('model/features.csv')
#generalize data
#train features
cv = CountVectorizer()
count_matrix = cv.fit_transform(df['overview'])
clean_df  = pd.read_csv('model/titles.csv')
#define function
def cosinesim(name):
    sim = cosine_similarity(count_matrix)
    name = name.lower()
    df['title']=df['title'].str.lower()
    if name not in df['title'].unique():
        return ('Try Aboslute Spelling or it may not be in our database')
    else:
        i = df.loc[df['title']==name].index[0]
        mov_id = list(enumerate(sim[i]))
        mov_id = sorted(mov_id,key = lambda x:x[1],reverse=True)
        mov_id = mov_id[1:11]
        rec_mov = []
        for i in range(len(mov_id)):
            a = mov_id[i][0]
            rec_mov.append(clean_df['title'][a])
        return rec_mov
def euclidsim(name):
    sim = euclidean_distances(count_matrix)
    name = name.lower()
    df['title']=df['title'].str.lower()
    if name not in df['title'].unique():
        return ('Try Aboslute Spelling or it may not be in our database')
    else:
        i = df.loc[df['title']==name].index[0]
        mov_id = list(enumerate(sim[i]))
        mov_id = sorted(mov_id,key = lambda x:x[1],reverse=False)
        mov_id = mov_id[1:11]
        rec_mov = []
        for i in range(len(mov_id)):
            a = mov_id[i][0]
            rec_mov.append(clean_df['title'][a])
        return rec_mov
print(cosinesim("toy story"))

['The Helix... Loaded', 'Chairman of the Board', 'Chiamatemi Francesco - Il Papa della gente', 'Food Chains', 'Khiladi 786', 'Scott Walker: 30 Century Man', 'To Be Frank, Sinatra at 100', 'Anomalisa', 'Invasion U.S.A.', 'Lovely, Still']
